In [1]:
parent_dir = joinpath(@__DIR__, "..")
include(joinpath(parent_dir, "helper_split.jl"))
include(joinpath(parent_dir, "algorithms.jl"))
include(joinpath(parent_dir, "helper_tt.jl"))

using Distributions, ForwardDiff, Plots

Set up the target, a mixture of 13 standard normals with equal weights

In [2]:
dim = 2
μ = Vector{Vector{Float64}}()
push!(μ, zeros(dim))
push!(μ, -10*ones(dim))
push!(μ, [10.,10.])
push!(μ, [10.,-10.])
push!(μ, [-10.,10.])
push!(μ, [20.,0.])
push!(μ, [-20.,0.])
push!(μ, [0.,-20.])
push!(μ, [0.,20.])
push!(μ, [20.,20.])
push!(μ, [20.,-20.])
push!(μ, [-20.,20.])
push!(μ, [-20.,-20.])
nr_mixtures = length(μ)
λ = 1/nr_mixtures
correlation = 0
Σ = correlation*ones(dim,dim) + (1-correlation)I
Σ_inv = inv(Σ)
# mixtures = [gaussian_pdf(i,Σ, Σ_inv) for i ∈ μ]
mixtures = [gaussian_pdf(μ[i],Σ, Σ_inv) for i in eachindex(μ)]
# mixtures = [gaussian_pdf(μ[i],Σ, Σ_inv) for i = 1:nr_mixtures]

function target(x::Vector)
    fn = 0
    for i = 1:nr_mixtures
        fn = fn + λ * mixtures[i](x)
    end
    fn
end

U(x)  = -log(target(x))
∇U(x) = ForwardDiff.gradient(U,x);

Plot the target distribution

In [ ]:
x = range(-50, stop = 50, length = 1000)
y = range(-35, stop = 35, length = 1000)
contour(x, y, (x, y) -> (target([x,y])),color=:plasma,ratio=:equal)

# contour!(x, y, (x, y) -> (target([x,y]))^(1-α))
# savefig(string("levelcurves_gaussian_mixture_alpha_",α,".pdf"))

Choose the speed function and define the base target distribution

In [75]:
α = 0.9
s(x)  = exp(α*U(x))
∇U_s(x) = (1-α)*∇U(x)
unnorm_target_s(x) = s(x) * exp(-U(x));

We consider two alternatives:

1. Simulate a path of the continuous-time ZZP targeting the base distribution, then discretise it, build a time-changed jump process, and finally discretise it.

2. Simulate the Metropolis-adjusted ZZP (Bertazzi, Dobson, Monmarché (2023)) targeting the base distribution, build a time-changed jump process, and finally discretise it.

We start with the first option.

First, we simulate a ZZP targeting the base distribution.

In [ ]:
m = minimum(μ)
M = maximum(μ)
Hessian_bound_alpha = diagm(((1-α) * (1 .+ 0.25 * (M.-m).^2)));   #computational bounds to simulate the ZZP targeting the base distribution

T = 3 * 10^4
x_init = zeros(dim)
v_init = rand((-1,1),dim)
skele = ZigZag(∇U_s, Hessian_bound_alpha, T, x_init,v_init)

pos1 = [skele[i].position[1] for i in eachindex(skele)]
pos2 = [skele[i].position[2] for i in eachindex(skele)]
p_std = plot(pos1,pos2, label = "", lc = "red",lw = 1.5, 
                xlabel = L"x_1", ylabel = L"x_2",
                xlim = [-50,50], ratio=:equal, grid=:none, 
                )
# p_std = scatter!(Tuple.(μ), label="Modes", markershape=:diamond, markercolor="green",legend=:bottomright)
display(p_std)
savefig("zzp_gaussmixture.pdf")

Then, we discretise the trajectories.

In [ ]:
delta = 2.
discr_chain = discretise_from_skeleton(skele,delta)
display(scatter(Tuple.(discr_chain),label ="",markersize=:2, color=:"red",
        # alpha=:0.25,
        markerstrokewidth=0,
        xlabel = L"x_1", ylabel = L"x_2",
        xlim = [-50,50], ratio=:equal, grid=:none, ))
# display(scatter!(Tuple.(μ), label="Modes", markershape=:diamond, markercolor="green",legend=:bottomright))
savefig("zzp_discretised_mixture.pdf")

Finally, we build the time-changed jump process with the discretised output and finally discretise the jump process.

In [ ]:
skele_jump = make_into_jumpproc(discr_chain, s)
delta_jump = .005
discr_jump = discretise_jumpprocess(skele_jump,delta_jump)
display(scatter(Tuple.(discr_jump),label ="",markersize=:2, color=:"red",
            alpha=:0.25,
            markerstrokewidth=0,
            xlabel = L"x_1", ylabel = L"x_2",
            xlim = [-50,50], ratio=:equal, grid=:none, ))
# display(scatter!(Tuple.(μ), label="Modes", markershape=:diamond, markercolor="green",legend=:bottomright))
savefig("TC-zzp_mixture.pdf")

Now, we focus on the second approach.

In [ ]:
δ = .5
N = 5 * 10^4
x_init = zeros(dim)
v_init = rand((-1,1),2)
chain = zzs_metropolis_randstepsize(unnorm_target_s,∇U_s,δ,N,x_init,v_init)
scatter(Tuple.([ch.position for (_,ch) in enumerate(chain)]),label ="",markersize=:1, color=:"red",markerstrokewidth=0,
        xlabel = L"x_1", ylabel = L"x_2",
        xlim = [-50,50], ratio=:equal, grid=:none, )
display(scatter!(Tuple.(μ), label="Modes", markershape=:diamond, markercolor="green",legend=:bottomright))
# savefig("metropolis_zzp_mixture.pdf")

Finally, we construct the time-changed jump process and discretise it.

In [ ]:
jump_proc = make_into_jumpproc(chain, s)
delta_jump = 0.02
discr_jump = discretise_jumpprocess(jump_proc,delta_jump)
display(scatter(Tuple.(discr_jump),label ="",markersize=:2, color=:"red",
            alpha=:0.25,
            markerstrokewidth=0,
            xlabel = L"x_1", ylabel = L"x_2",
            xlim = [-50,50], ratio=:equal, grid=:none, ))
# display(scatter!(Tuple.(μ), label="Modes", markershape=:diamond, markercolor="green",legend=:bottomright))
# savefig("TC-metropolis_zzp_mixture.pdf")

Now we compare it to the standard Metropolis-adjusted ZZP, i.e. setting s(x)=1.

In [ ]:
α = 0.
s(x)  = 1
∇U_s(x) = ∇U(x)
unnorm_target_s(x) = exp(-U(x))

δ = .5
N = 5 * 10^4
x_init = zeros(dim)
v_init = rand((-1,1),2)
chain = zzs_metropolis_randstepsize(unnorm_target_s,∇U_s,δ,N,x_init,v_init)
jump_proc = make_into_jumpproc(chain, s)
delta_jump = 20.
discr_jump = discretise_jumpprocess(jump_proc,delta_jump)
display(scatter(Tuple.(discr_jump),label ="",markersize=:2, color=:"red",
            alpha=:0.15,
            markerstrokewidth=0,
            xlabel = L"x_1", ylabel = L"x_2",
            xlim = [-50,50], ratio=:equal, grid=:none, ))
# savefig("jump_metropolis_zzp_mixture.pdf")